# Phi-4 Mini ONNX Parallel Function Calling Tutorial

ဒီနိုတ်ဘွတ်ခ်မှာ Phi-4 Mini ကို ONNX Runtime GenAI နဲ့အတူ အသုံးပြုပြီး ပျော်လည်း Parallel Function Calling ကို ဘယ်လိုလုပ်ရမလဲဆိုတာကို ပြသပေးမှာဖြစ်ပါတယ်။ Function Calling က မော်ဒယ်ကို အသုံးပြုသူရဲ့ တောင်းဆိုမှုအပေါ်မူတည်ပြီး အပြင်အဆင့်ကိရိယာတွေ၊ API တွေကို ထိထိရောက်ရောက် ခေါ်သုံးနိုင်စေပါတယ်။

## အနှစ်ချုပ်

ဒီသင်ခန်းစာမှာ သင်လေ့လာနိုင်မယ့်အရာတွေက:
- Phi-4 Mini ကို ONNX Runtime GenAI နဲ့အတူ စတင်အသုံးပြုနည်း
- လေယာဉ်လက်မှတ်နဲ့ ဟိုတယ်ဘွတ်ကင်လုပ်ဖို့ Function Schema တွေ သတ်မှတ်နည်း
- Lark Grammar ကို အသုံးပြုပြီး Guided Generation နဲ့ ဖွဲ့စည်းထားတဲ့ output ရယူနည်း
- ခက်ခဲတဲ့ ခရီးသွားဘွတ်ကင်အခြေအနေတွေမှာ Parallel Function Calling ကို အကောင်အထည်ဖော်နည်း

## လိုအပ်ချက်များ

ဒီနိုတ်ဘွတ်ခ်ကို အကောင်အထည်ဖော်မယ့်အခါမှာ သင်မှာ ရှိထားဖို့လိုတာတွေက:
- Phi-4 Mini ONNX မော်ဒယ်ကို ဒေါင်းလုပ်လုပ်ပြီးဖြစ်ရမယ်
- `onnxruntime-genai` package ကို ထည့်သွင်းပြီးဖြစ်ရမယ်
- Function Calling အကြောင်းအရာတွေကို အခြေခံနားလည်မှု ရှိထားရမယ်


## အဆင့် ၁ - လိုအပ်သောစာကြည့်တိုက်များကို သွင်းယူပါ

ပထမဦးစွာ၊ function calling ကို အကောင်အထည်ဖော်ရန် လိုအပ်သောစာကြည့်တိုက်များကို သွင်းယူပါမည်။


In [1]:
import json

In [2]:
import onnxruntime_genai as og

## အဆင့် ၂ - မော်ဒယ်တပ်ဆင်ခြင်းနှင့် ပြင်ဆင်မှု

ယခုအချိန်တွင် Phi-4 Mini ONNX မော်ဒယ်ကို ပြင်ဆင်သတ်မှတ်ပါမည်။ မော်ဒယ်ရှိတဲ့ ဖိုင်လမ်းကြောင်းကို သင့်ရဲ့ အမှန်တကယ် မော်ဒယ်ဒိုင်ရက်ထဲက လမ်းကြောင်းနဲ့ အစားထိုးပေးပါ။


In [ ]:
# TODO: Replace with your actual Phi-4 Mini ONNX model path
# Download from: https://huggingface.co/microsoft/Phi-4-mini-onnx
path = 'Your phi-4-mini-onnx path'  # Update this path!

In [4]:
config = og.Config(path)

In [5]:
model = og.Model(config)

In [6]:
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

## အဆင့် ၃ - ထုတ်လုပ်မှု ပarameters များကို ပြင်ဆင်ပါ

မော်ဒယ်၏ output အပြုအမူကို ထိန်းချုပ်ရန် ထုတ်လုပ်မှု parameters များကို ပြင်ဆင်ပါ။ ဒီဆက်တင်များက function calling အတွက် သေချာပြီး အာရုံစိုက်ထားသော အဖြေများကို အာမခံပေးပါသည်။


In [7]:
# Configure generation parameters for deterministic function calling
search_options = {}
search_options['max_length'] = 4096      # Maximum tokens to generate
search_options['temperature'] = 0.00001  # Very low temperature for deterministic output
search_options['top_p'] = 1.0            # Nucleus sampling parameter
search_options['do_sample'] = False       # Disable sampling for consistent results

## အဆင့် ၄ - ရရှိနိုင်သော လုပ်ဆောင်ချက်များ သတ်မှတ်ရန်

ဒီအဆင့်မှာ ကျွန်တော်တို့ရဲ့ AI အကူအညီပေးစနစ်က အသုံးပြုနိုင်မည့် လုပ်ဆောင်ချက်များကို သတ်မှတ်ပေးပါမယ်။ ဥပမာအနေနဲ့ ဒီမှာ ခရီးသွားရေးနှင့်ဆိုင်သော လုပ်ဆောင်ချက်နှစ်ခုရှိပါတယ် -

1. **booking_flight_tickets**: လေဆိပ်များအကြား လေယာဉ်လက်မှတ်များ ကြိုတင်မှာယူရန်
2. **booking_hotels**: ဟိုတယ်အဆောင်များ ကြိုတင်မှာယူရန်

လုပ်ဆောင်ချက်များကို OpenAI ရဲ့ function calling schema ပုံစံအတိုင်း သတ်မှတ်ထားပါတယ်။


In [8]:
tool_list = '[{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport code", "type": "string"}, "destination_airport_code": {"description": "The name of Destination airport code", "type": "string"}, "departure_date": {"description": "The date of outbound flight", "type": "string"}, "return_date": {"description": "The date of return flight", "type": "string"}}}, {"name": "booking_hotels", "description": "booking hotel", "parameters": {"destination": {"description": "The name of the city", "type": "string"}, "check_in_date": {"description": "The date of check in", "type": "string"}, "checkout_date": {"description": "The date of check out", "type": "string"}}}]'

## အဆင့် ၅ - သဒ္ဒါဖန်တီးရေးအကူအညီပေးသော Function များ

ဒီအကူအညီပေးသော Function များက ကျွန်ုပ်တို့၏ Function အဓိပ္ပာယ်ဖော်ပြချက်များကို Lark သဒ္ဒါပုံစံအဖြစ် ပြောင်းလဲပေးပြီး ONNX Runtime GenAI မှ အညွှန်းအရ ဖန်တီးမှုများအတွက် အသုံးပြုသည်။ ဒါက မော်ဒယ်မှ သင့်တော်သော JSON ပုံစံဖြင့် Function ခေါ်ဆိုမှုများကို ထုတ်ပေးနိုင်ရန် သေချာစေသည်။


In [9]:
def get_lark_grammar(input_tools):
    tools_list = get_tools_list(input_tools)
    prompt_tool_input = create_prompt_tool_input(tools_list)
    if len(tools_list) == 1:
        # output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(tools_list[0]))
        output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(convert_tool_to_grammar_input(tools_list[0])))
        return prompt_tool_input, output
    else:
        return prompt_tool_input, "start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {\"anyOf\": [" + ','.join([json.dumps(tool) for tool in tools_list]) + "]}"


In [10]:
def get_tools_list(input_tools):
    # input_tools format: '[{"name": "fn1", "description": "fn details", "parameters": {"p1": {"description": "details", "type": "string"}}},
    # {"fn2": 2},{"fn3": 3}]'
    tools_list = []
    try:
        tools_list = json.loads(input_tools)
    except json.JSONDecodeError:
        raise ValueError("Invalid JSON format for tools list, expected format: '[{\"name\": \"fn1\"},{\"name\": \"fn2\"}]'")
    if len(tools_list) == 0:
        raise ValueError("Tools list cannot be empty")
    return tools_list

In [11]:
def create_prompt_tool_input(tools_list):
    tool_input = str(tools_list[0])
    for tool in tools_list[1:]:
        tool_input += ',' + str(tool)
    return tool_input

In [12]:
def convert_tool_to_grammar_input(tool):
    param_props = {}
    required_params = []
    for param_name, param_info in tool.get("parameters", {}).items():
        param_props[param_name] = {
            "type": param_info.get("type", "string"),
            "description": param_info.get("description", "")
        }
        required_params.append(param_name)
    output_schema = {
        "description": tool.get('description', ''),
        "type": "object",
        "required": ["name", "parameters"],
        "additionalProperties": False,
        "properties": {
            "name": { "const": tool["name"] },
            "parameters": {
                "type": "object",
                "properties": param_props,
                "required": required_params,
                "additionalProperties": False
            }
        }
    }
    if len(param_props) == 0:
        output_schema["required"] = ["name"]
    return output_schema

In [13]:
get_lark_grammar(tool_list)

("{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}",
 'start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {"anyOf": [{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport c

## အဆင့် ၆ - သဒ္ဒါဖွဲ့စည်းမှု စမ်းသပ်ခြင်း

သဒ္ဒါဖွဲ့စည်းမှုလုပ်ဆောင်မှုများကို စမ်းသပ်ကြည့်ပြီး၊ ကျွန်ုပ်တို့၏ tool အဓိပ္ပာယ်ဖော်ပြချက်များကို သင့်လျော်သောပုံစံသို့ ပြောင်းလဲပုံကို ကြည့်ရှုကြမည်။


In [14]:
prompt_tool_input, guidance_input = get_lark_grammar(tool_list)

## အဆင့် ၇: စနစ်အမိန့်နှင့် မျိုးစေ့ထုတ်လုပ်သူကို ပြင်ဆင်ပါ

ယခုအချိန်တွင် မော်ဒယ်ကို အသုံးပြုနိုင်သော ကိရိယာများအကြောင်း ပြောပြသည့် စနစ်အမိန့်ကို ဖန်တီးပြီး မျိုးစေ့ထုတ်လုပ်မှုဆိုင်ရာ လမ်းညွှန်သတ်မှတ်ချက်များနှင့်အတူ မျိုးစေ့ထုတ်လုပ်သူကို ပြင်ဆင်သတ်မှတ်ပါ။


In [15]:
# Define the system prompt that introduces the assistant and its capabilities
system_prompt = "You are a helpful assistant with these tools."

In [16]:
# Format the system message with tools information
messages = f"""[{{"role": "system", "content": "{system_prompt}", "tools": "{prompt_tool_input}"}}]"""

In [17]:
# Apply chat template to format the system prompt properly
tokenizer_input_system_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=False)

In [18]:
tokenizer_input_system_prompt

"<|system|>You are a helpful assistant with these tools.<|tool|>{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}<|/tool|><|end|><|endoftext|>"

In [19]:
input_tokens = tokenizer.encode(tokenizer_input_system_prompt)

In [20]:
input_tokens = input_tokens[:-1]

In [21]:
system_prompt_length = len(input_tokens)

## အဆင့် ၈: Guided Generation ဖြင့် Generator ကို စတင်မောင်းနှင်ပါ

ယခုအချိန်တွင် ကျွန်ုပ်တို့၏ သတ်မှတ်ထားသော parameters များနှင့်အတူ generator ကို ဖန်တီးပြီး Guided Generation အတွက် Lark grammar ကို အသုံးပြုမည်ဖြစ်သည်။


In [22]:
# Create generator parameters and apply search options
params = og.GeneratorParams(model)
params.set_search_options(**search_options)

In [23]:
# Apply Lark grammar for guided generation to ensure valid function call format
params.set_guidance("lark_grammar", guidance_input)

In [24]:
generator = og.Generator(model, params)

In [25]:
generator.append_tokens(input_tokens)

## အဆင့် ၉ - ပူးတွဲလုပ်ဆောင်မှု ဖန်ရှင်ခေါ်ဆိုမှုကို စမ်းသပ်ပါ

အခုတော့ အများဖက်စီးပွားခရီးသွားအခြေအနေတစ်ခုကို စမ်းသပ်ကြမယ်၊ ဒါဟာ ဖန်ရှင်အများကြီးကို ခေါ်ဆိုရမယ့် အခြေအနေဖြစ်ပါတယ်။


In [26]:
# Complex travel booking request that requires both flight and hotel booking
text = "book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris"

In [27]:
# Format user message and apply chat template
messages = f"""[{{"role": "user", "content": "{text}"}}]"""

# Apply Chat Template for user input
user_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=True)
input_tokens = tokenizer.encode(user_prompt)
generator.append_tokens(input_tokens)

In [28]:
user_prompt

'<|user|>book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris<|end|><|assistant|>'

### ဖန်တီးမှုလုပ်ဆောင်မှုများကို ဖန်တီးပါ

မော်ဒယ်သည် ယခုအခါ အသုံးပြုသူ၏ တောင်းဆိုမှုအရ ဖွဲ့စည်းထားသော လုပ်ဆောင်မှုများကို ဖန်တီးပေးမည်ဖြစ်သည်။ ဦးတည်ချက်ထားပြီး ဖန်တီးမှုကြောင့် အထွက်အဖြစ် JSON ပုံစံမှန်ကန်စွာ ထွက်ရှိမည်ဖြစ်ပြီး တိုက်ရိုက်အကောင်အထည်ဖော်နိုင်ပါသည်။

**မျှော်မှန်းထားသော အထွက်**: မော်ဒယ်သည် အောက်ပါလုပ်ဆောင်မှုများအတွက် ဖန်တီးမှုများကို ထုတ်ပေးရမည်ဖြစ်သည် -
1. Beijing (PEK) မှ Paris (CDG) သို့ သွားရန်အတွက် `booking_flight_tickets`
2. Paris တွင် နေရာချထားရန်အတွက် `booking_hotels`

အောက်ပါ cell ကို run လုပ်ပြီး တိုက်ရိုက်ဖန်တီးမှုကို ကြည့်ရှုပါ:


In [29]:
# Generate tokens one by one and stream the output
while not generator.is_done():
    generator.generate_next_token()
    new_token = generator.get_next_tokens()[0]
    print(tokenizer_stream.decode(new_token), end='', flush=True)

[{"name": "booking_flight_tickets", "arguments": {"origin_airport_code": "PEKK", "destination_airport_code": "CDG", "departure_date": "2025-12-04", "return_date": "2025-12-10"}}, {"name": "booking_hotels", "arguments": {"destination": "Paris", "check_in_date": "2025-12-04", "checkout_date": "2025-12-10"}}]

## နိဂုံး

🎉 **ဂုဏ်ယူပါတယ်!** သင် ONNX Runtime GenAI ကို အသုံးပြု၍ Phi-4 Mini ဖြင့် ပူးတွဲလုပ်ဆောင်မှုကို အောင်မြင်စွာ အကောင်အထည်ဖော်နိုင်ခဲ့ပါပြီ။

### သင်လေ့လာခဲ့တာများ:

1. **မော်ဒယ် Setup**: ONNX Runtime GenAI ဖြင့် Phi-4 Mini ကို စနစ်တကျ ပြင်ဆင်ပုံ
2. **လုပ်ဆောင်မှု အဓိပ္ပာယ်ဖော်ပြခြင်း**: AI function calling အတွက် function schemas ကို သတ်မှတ်ပုံ
3. **Guided Generation**: Lark grammar ကို အသုံးပြု၍ ဖွဲ့စည်းထားသော output ကို ထုတ်လုပ်ပုံ
4. **ပူးတွဲလုပ်ဆောင်မှုများ**: လိုအပ်သော အများအပြား function calls ကို စနစ်တကျ ကိုင်တွယ်ပုံ

### အဓိက အကျိုးကျေးဇူးများ:

- ✅ **ဖွဲ့စည်းထားသော Output**: Guided generation က သက်ဆိုင်သော JSON function calls ကို အတိအကျ ထုတ်ပေးနိုင်ခြင်း
- ✅ **ပူးတွဲလုပ်ဆောင်မှု**: တစ်ခါတည်းမှာ function calls အများအပြားကို ကိုင်တွယ်နိုင်ခြင်း
- ✅ **မြင့်မားသော စွမ်းဆောင်ရည်**: ONNX Runtime က အကောင်းဆုံး inference ကို ပေးနိုင်ခြင်း
- ✅ **အလွယ်တကူ ပြင်ဆင်နိုင်သော Schema**: Function definitions ကို အလွယ်တကူ ထည့်သွင်းခြင်းနှင့် ပြင်ဆင်နိုင်ခြင်း

### ရင်းမြစ်များ:

- [Phi-4 Mini Documentation](https://huggingface.co/microsoft/Phi-4-mini-onnx)
- [ONNX Runtime GenAI Documentation](https://onnxruntime.ai/docs/genai/)
- [Function Calling Best Practices](https://platform.openai.com/docs/guides/function-calling)



---

**ဝက်ဘ်ဆိုက်မှတ်ချက်**:  
ဤစာရွက်စာတမ်းကို AI ဘာသာပြန်ဝန်ဆောင်မှု [Co-op Translator](https://github.com/Azure/co-op-translator) ကို အသုံးပြု၍ ဘာသာပြန်ထားပါသည်။ ကျွန်ုပ်တို့သည် တိကျမှန်ကန်မှုအတွက် ကြိုးစားနေပါသော်လည်း၊ အလိုအလျောက်ဘာသာပြန်ဆိုမှုများတွင် အမှားများ သို့မဟုတ် မတိကျမှုများ ပါဝင်နိုင်သည်ကို ကျေးဇူးပြု၍ သတိပြုပါ။ မူရင်းစာရွက်စာတမ်းကို ၎င်း၏ မူလဘာသာစကားဖြင့် အာဏာတည်သောရင်းမြစ်အဖြစ် သတ်မှတ်ရန် လိုအပ်ပါသည်။ အရေးကြီးသော အချက်အလက်များအတွက် လူကောင်းမွန်သော ပရော်ဖက်ရှင်နယ်ဘာသာပြန်ကို အကြံပြုပါသည်။ ဤဘာသာပြန်ကို အသုံးပြုခြင်းမှ ဖြစ်ပေါ်လာသော နားလည်မှုမှားများ သို့မဟုတ် အဓိပ္ပါယ်မှားများအတွက် ကျွန်ုပ်တို့သည် တာဝန်မယူပါ။
